# 2D CCZ Protocol

## Hexagonal Lattice Construction
We construct a hexagonal lattice as follows.
We start with a square $L\times L$ lattice.
For each square with coordinates $(a,b)$ place three vertices:
$(0,a,b),(1,a,b)$ and $(2,a,b)$ where the first co-ordinate indicates the colour of the vertex.

For each vertex $v = (c,a,b)$, directed edges are defined by the $\text{Succ}(v,t)$ function. The parameter $t$ denotes one of three directions and $L$ is the lattice size:
- $\text{Succ}((c,a,b),0) = (c + 1, a + d, b + d)$: right, up in diagram
- $\text{Succ}((c,a,b),1) = (c + 1, a – 1 + d, b + d)$: left, up
- $\text{Succ}((c,a,b),2) = (c + 1, a + d, b - 1 + d)$: right, down

where $d := 1$ if $c = 2$ else 0. 
Addition for colours is modulo $3$ and for lattice coordinates modulo $L$.
The vertex $\text{Succ}(v,t)$ has colour $c+1$ and the colour of the edge $(v,\text{Succ}(v,t))$ is $c-1$ (as it joins a vertex of colour $c$ to one of colour $c+1$).
We also define the function $\text{Pred}(v,t)$ which is the inverse of $\text{Succ}$.

## Toric Code Construction
We place qubits on edges of the hexagonal lattice and we define a toric code for each colour $c$ as follows:
- Z Checks: weight three operators of two types: 
    1. For each vertex $v$ of colour $c+1$: $\prod_{0 \le t \le 2}Z_{(v,\text{Succ}(v,t))}$; and
    2. For each vertex $v$ of colour $c-1$:  $\prod_{0 \le t \le 2}Z_{(\text{Pred}(v,t),v)}$.
- X Checks: weight six operators for each vertex $v$ of colour $c$: $\prod_{s \ne t}X_{(\text{Succ}(v,t), \text{Pred}(v,s)))}$.

Each of the toric codes have two logical qubits labelled $(c,0)$ and $(c,1)$

![hex 2D lattice](./hex_lattice.jpeg)

In [49]:
import add_parent_dir
from common import *
from XPAlgebra import *
from XPCodes import *
import numpy as np
import itertools as iter

def Succ(v,t,L):
    '''Generate successor of vertex v of type t and square lattice size L'''
    ## number of dimensions - potential to generalise this
    D = 3
    ## convert tupe to list
    v = list(v)
    ## Decrement location index depending on type t
    if t > 0:
        v[t] = (v[t] - 1) % L
    ## Increment both location indices when vertex colour is D-1
    if v[0] == D-1:
        for i in [1,2]:
            v[i] = (v[i] + 1) % L
    ## increment colour
    v[0] = (v[0] + 1) % D 
    ## convert list to tuple
    return tuple(v)  

def Pred(v,t,L):
    '''Generate predecessor of vertex v of type t and square lattice size L'''
    ## number of dimensions - potential to generalise this
    D = 3
    ## convert tupe to list
    v = list(v)
    ## Increment location index depending on type t
    if t > 0:
        v[t] = (v[t] + 1) % L
    ## Decrement both location indices when vertex colour is 0
    if v[0] == 0:
        for i in [1,2]:
            v[i] = (v[i] - 1) % L
    ## decrement colour
    v[0] = (v[0] - 1) % D 
    ## convert list to tuple
    return tuple(v)  

def hex_surface(L):
    D = 3
    ## Generate vertices
    V = [(c,i,j) for c in range(D) for j in range(L) for i in range(L)]
    # VDict = {V[i]: i for i in range(len(V))}
    ## rearrange order of V to ensure correct colour of edges
    nV = len(V) // D
    ix = np.hstack([np.arange(nV,D * nV),np.arange(nV)])
    ## generate edges
    E = [(V[i],Succ(V[i],t,L)) for i in ix for t in range(D) ]
    EDict = {E[i]: i for i in range(len(E))}
    nE = len(E) // D
    SXcc = [[] for i in range(D)]
    SZcc = [[] for i in range(D)]
    CZcc = [[] for i in range(D)]
    for i in range(len(V)):
        v = V[i]
        cV = i // nV
        szSucc = [(v,Succ(v,t,L)) for t in range(D)]
        szPred = [(Pred(v,t,L),v) for t in range(D)]
        sx = [(Succ(v,t1,L),Succ(Succ(v,t1,L),t2,L)) for (t1,t2) in iter.permutations(range(D),2)]
        cz = [[(v,Succ(v,t1,L)),(Succ(Succ(v,t1,L),t2,L),v)] for (t1,t2) in iter.permutations(range(D),2)]
        ## Convert to edge indices and update SX,SZ,CZ
        ## szSucc is of colour cV-1
        SZcc[(cV-1) % D].append([EDict[a] for a in szSucc])    
        ## szPred is of colour cV+1
        SZcc[(cV+1) % D].append([EDict[a] for a in szPred])  
        ## sx and cz are same colour as v
        SXcc[cV].append([EDict[a] for a in sx])
        CZcc[cV].append([(EDict[a],EDict[b]) for (a,b) in cz])
    return SXcc, SZcc, CZcc, 3 * nE

def getKM(SX, LX,t):
    '''Return KM - Z_N matrix representing phase and z components of diagonal logical identities.'''
    r,n = SX.shape
    Eq = ZMatZeros((1,n))
    A = Orbit2dist(Eq,np.vstack([SX,LX]), t)
    A = np.hstack([A,[[1]]*len(A)])
    return getKer(A,1<<t)

def printXS(A):
    '''print compact rep of XS operators'''
    p,x,z = XPcomponents(A)
    temp = []
    # phaseArr = ["","i","-1","-i"]
    # temp.append(phaseArr[p//2])
    if np.sum(x) > 0:
        a = ['X:'] + [str(i) for i in bin2Set(x)]
        temp.append(" ".join(a))
    zArr = ['',"S:","Z:","S3:"]
    if np.sum(z) > 0:
        Zdict = {i:[] for i in range(4)}
        for i in range(len(z)):
            Zdict[z[i]].append(i)
        for j in range(1,4):
            if len(Zdict[j]) > 0:
                a = [zArr[j]] + [str(i) for i in Zdict[j]]
                temp.append(" ".join(a))
    return "\n".join(temp)

## Form stabilisers of toric code and CZ operator on hex surface on LxL grid
L = 2
SX, SZ, CZ, n = hex_surface(L)

## Initial code: toric code on colour 0, 1
Sxx = Sets2ZMat(n, SX[0] + SX[1])
## add Z stabiliser on each qubit of colour 2
Szz = Sets2ZMat(n, SZ[0] + SZ[1] + [[i] for i in range(2*n//3,n)])
## RREF of X-checks and Z-checks
Hx, rowops = How(Sxx,2)
Hz, rowops = How(Szz,2)
## logical X of the toric codes
Lxx = PSFgetLXx(Hx,Hz)

## get embedded code
## physical qubits
V = [[i] for i in range(n)] 
## pairs of qubits involved in CZ operators
V += [s for CZi in CZ for cz in CZi for s in cz ]
VDict = {tuple(V[i]):i for i in range(len(V))}
V = Sets2ZMat(n, V)
M = V[n:]
nV = len(V)

Sxx = matMul(Sxx,V.T,2)
Lxx = matMul(Lxx,V.T,2)
## Z-stabilisers of embedded code
Szz1 = np.hstack([Szz,ZMatZeros((len(Szz),nV-n))])
Szz1 = np.vstack([Szz1,np.hstack([M,ZMatI(len(M))])])
Szz = getKer(np.vstack([Sxx,Lxx]),2)
# print('Szz1',freqTablePrint(np.sum(Szz1,axis=-1)))

## Initialise in |++++> logical state - add Lxx
Sx = makeXP(0,np.vstack([Sxx,Lxx]),0)
## Or just initialse using Sxx
# Sx = makeXP(0,Sxx,0)
Sz = makeXP(0,0,Szz)
C = Code(np.vstack([Sx,Sz]),2)
CW = C.getCodewords()
# CW = [XPSetN(S,2,N) for S in CW]

## Logical CZ Operator
Define the operator $CZ_c := \prod_{v=(c,a,b)}\prod_{s \ne t} CZ_{(v,\text{Succ}(v,t)), (\text{Pred}(v,s),v)}$.
This operator acts qubits of colour $c\pm 1$ and is a logical operator on the code formed by combining the surface codes of colours $c\pm 1$. The logical action of $CZ_c$ is $CZ_{(c-1,0),(c+1,1)}CZ_{(c-1,1),(c+1,0)}$.

For instance, when applied to the toric codes of colour $0$ and $1$, the operator $CZ_2$ has logical action $CZ_{03}CZ_{12}$ as illustrated below.

In [50]:
## Calculate CZ logical operator and Star Operators
c = 2
SXc = Sets2ZMat(n,SX[c])
Mx = matMul(SXc,V.T,2)
Mz = []

N = 4
for cz in CZ[c]:
    z = ZMatZeros(nV)
    for (a,b) in cz:
        z[VDict[(a,)]] += 3
        z[VDict[(b,)]] += 3
        z[VDict[(a,b)]] += 1
    Mz.append(np.mod(z,N))
CZLO = np.mod(np.sum(Mz,axis=0),N)
# print('CZLO',ZmatPrint(CZLO))

k = len(Lxx)
Lxx1 = np.hstack([ZMatI(k),matMul(Lxx,V.T,2)])
Eq = ZMatZeros((1,len(V)+k))
A = Orbit2dist(Eq,Lxx1,4)
LI, Em = A[:,:k],A[:,k:]
CZphase = matMul(CZLO,Em.T,N)[0]
print('Phase applied by CZ_2 on toric code on colours 0, 1')
for i in range(len(Em)):
    print(f'{"-" if CZphase[i] == 2 else "+"}1 |{ZMat2str(LI[i])}>')

Phase applied by CZ_2 on toric code on colours 0, 1
+1 |0000>
+1 |1000>
+1 |0100>
+1 |0010>
+1 |0001>
+1 |1100>
+1 |1010>
-1 |1001>
-1 |0110>
+1 |0101>
+1 |0011>
-1 |1110>
-1 |1101>
-1 |1011>
-1 |0111>
+1 |1111>


## Preparation of Twisted Quantum Double Phase
1. Prepare toric codes of colour 0 and 1 in the logical $\ket{++++}$ state.
2. Qubits of colour 2 are prepared in the $\ket{0}$ state.
3. The logical action of $CZ_2$ is $\overline{CZ_{03}CZ_{12}}$ so this applies a phase of $-1$ on 6 of the 16 logical basis states and $+1$ on the remaining 10 basis states;
2. We measure star operators of the colour 2 toric code in sequence. For each vertex $v$ of colour $2$, these are of form:
$\prod_{s \ne t}X_{(\text{Succ}(v,t), \text{Pred}(v,s)))}CZ_{(v,\text{Succ}(v,t)), (\text{Pred}(v,s),v)}$

Note that the product of the star operators is the same as the product of the X-checks of the colour 2 toric code and  $CZ_2$
Measurements are done by constructing the embedded code which allows us to represent CZ operators as XS operators. The method for doing this is set out in https://arxiv.org/abs/1404.5327, https://arxiv.org/abs/2303.15615 and measurements of XS operators are performed as outlined in https://arxiv.org/abs/2203.00103 .

Up to but not including the final measurement of star operators, we obtain outcomes of +/-1 with 50% probability. 
For the final measurement, we obtain the following outcomes:
- 5/8 probability: non-additive code with 10 logical basis states correspoding to $\braket{CZ_2} = +1$
- 3/8 probability: non-additive code with 6 logical basis states corresponding to $\braket{CZ_2} = -1$

In the analysis below, measurements are performed then one of the outcome states is chosen at random for the next measurement stage.

In [51]:
## Prepare TQD by measuring star operators of colour 2
P = 2*N
M = makeXP(0,Mx,Mz)
for i in range(len(M)):
    print('\n#######################################################################################')
    print(f'Step {i} Measuring {XP2Str(M[i],N)}')
    print(printXS(M[i]))
    print('#######################################################################################')
    L, p1, CW1 = MeasureCodewords(CW,M[i],N)
    ## choose random outcome o from measurment
    o = np.random.choice(range(len(L)))
    for i in range(len(L)):
        if p1[i] > 0:
            print(f'\nMeasurement Outcome: w^{L[i]}/{2*N}')
            print(f'Probability: {p1[i]}')
            S, c = CW1[i][0]
            S = XPSetN(S,P,N)
            if i == o:
                CW = [S]    
                print('**Randomly selected outcome**')         
            Eq,Sx = cosetDecomposition(XPx(S))
            print('Number of logical states',len(Eq))
            CZphase = matMul(CZLO,S.T,N)
            print('Phases applied by CZ_2 to basis states in codewords',freqTablePrint(CZphase[0]))



#######################################################################################
Step 0 Measuring XP_4(0|000000000000000000000000000110101011000000111100110011001111111010101101010111000000000000000000000000000000|200020002000222000000000000000000000000000000000000000000000000000000000000000000000111111000000000000000000)
X: 27 28 30 32 34 35 42 43 44 45 48 49 52 53 56 57 58 59 60 61 62 64 66 68 69 71 73 75 76 77
S: 84 85 86 87 88 89
Z: 0 4 8 12 13 14
#######################################################################################

Measurement Outcome: w^0/8
Probability: 0.5
Number of logical states 1
Phases applied by CZ_2 to basis states in codewords 0:1280,2:768

Measurement Outcome: w^4/8
Probability: 0.5
**Randomly selected outcome**
Number of logical states 1
Phases applied by CZ_2 to basis states in codewords 0:1280,2:768

#######################################################################################
Step 1 Measuring XP_4(0|0000000000000000000000001100000

## Return to Toric Phase
Measure Z on each qubit of colour 2 to return to the toric phase:   
- First three measurements have outcomes +/-1 with 50% prob
- Subsequent measurements have deterministic outcomes
- Remains a non-additive code with either 6 or 10 logical basis states

In [52]:
## Return to Toric phase
## Z operators on each of the qubits of colour 2
Mx = 0
Mz = [set2Bin(nV,[i]) * N // 2 for i in range(2*n //3, n)]
M = makeXP(0,Mx,Mz)
for i in range(len(M)):
    print('\n#######################################################################################')
    print(f'Step {i} Measuring {XP2Str(M[i],N)}')
    print(printXS(M[i]))
    print('#######################################################################################')
    L, p1, CW1 = MeasureCodewords(CW,M[i],N)
    ## choose random outcome o from measurment
    o = np.random.choice(range(len(L)))
    for i in range(len(L)):
        if p1[i] > 0:

            print(f'\nMeasurement Outcome: w^{L[i]}/{2*N}')
            print(f'Probability: {p1[i]}')
            S, c = CW1[i][0]
            S = XPSetN(S,P,N)
            if i == o:
                CW = [S]      
                print('**Randomly Selected outcome**')       
            Eq,Sx = cosetDecomposition(XPx(S))
            print('Number of logical states',len(Eq))
            CZphase = matMul(CZLO,S.T,N)
            print('Phases applied by CZ_2 to basis states in codewords',freqTablePrint(CZphase[0]))



#######################################################################################
Step 0 Measuring XP_4(0|000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000|000000000000000000000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000)
Z: 24
#######################################################################################

Measurement Outcome: w^0/8
Probability: 0.5
Number of logical states 10
Phases applied by CZ_2 to basis states in codewords 0:2560

Measurement Outcome: w^4/8
Probability: 0.5
**Randomly Selected outcome**
Number of logical states 10
Phases applied by CZ_2 to basis states in codewords 0:2560

#######################################################################################
Step 1 Measuring XP_4(0|000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000|0000000000000000000000000200000000000000000000000000000000